In [5]:
import spacy
from spacy_ann import AnnLinker

# Load the spaCy model from the output_dir you used from the create_index command
model_dir = "models/ann_linker/"
nlp = spacy.load(model_dir)

# The NER component of the en_core_web_md model doesn't actually recognize the aliases as entities
# so we'll add a spaCy EntityRuler component for now to extract them.
ruler = nlp.create_pipe('entity_ruler')
patterns = [{"label": "SKILL", "pattern": alias} for alias in nlp.get_pipe('ann_linker').kb.get_alias_strings()]
ruler.add_patterns(patterns)
nlp.add_pipe(ruler, before="ann_linker")

ModuleNotFoundError: No module named 'spacy_ann'

In [ ]:
doc = nlp("NLP is a highly researched subset of Machine learning.")
[(e.text, e.label_, e.kb_id_) for e in doc.ents]

In [ ]:
doc.vector_norm

In [ ]:
import srsly
import numpy as np
entities = list(srsly.read_jsonl('data/entities.jsonl'))
natl_doc = nlp.make_doc(entities[2]['description'])
neur_doc = nlp.make_doc(entities[3]['description']) 

In [ ]:
entity_encodings = np.asarray([natl_doc.vector, neur_doc.vector])
entity_norm = np.linalg.norm(entity_encodings, axis=1)
entity_norm

In [ ]:
sims = np.dot(entity_encodings, doc.vector.T) / (doc.vector_norm * entity_norm)
sims.argmax()

In [ ]:
patterns = [
    {"label": "SKILL", "pattern": alias}
    for alias in nlp.get_pipe('ann_linker').kb.get_alias_strings()
]

In [ ]:
print([(e.text, e.label_, e.kb_id_) for e in doc.ents])

In [ ]:
nlp("More text about nlpe")

In [ ]:
ent = list(doc.ents)[0]

In [ ]:
getattr(ent._, None, 'default')